# The Rise and Fall of the Third Reich
# A History of Nazi Germany 
# by William L. Shirer
## A Textual Analysis

Read in file.

In [29]:
import pandas as pd
import book_reader as br
import numpy as np
from IPython.display import display
rafo3r_text = open('rafo3r.txt', 'r', encoding='utf-8').read()
generate_book_df = False
generate_toc_df = False
rafo3r_reader = br.book_reader('rafo3r', generate_book_df, generate_toc_df)
book_file = 'rafo3r.txt'
h5_file = 'rafo3r.h5'
rafo3r, toc = rafo3r_reader.main(book_file, h5_file)

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('yg2bsm', '8e3m3cer5e')

Import/setup Spacy.

Note: You must install spacy and download the 'en' model by running 'python -m spacy.en.download'. The download command fetches and installs about 500 MB of data which it installs within the spacy package directory.

In [2]:
import spacy
from spacy.attrs import ORTH

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load('en')

In [3]:
book = nlp(rafo3r) #this creates a spacy.tokens.doc.Doc object

In [ ]:
book_sents = book.sents
for i, s in enumerate(book_sents):
    if i > 1000: break
    #print (s)
print (book.ents)
ents = list(book.ents)
words = list(book)

In [28]:
#%matplotlib inline
germany = rafo3r[rafo3r['Words'] == 'germany']
#c,bins = np.histogram(germany)
#germany.hist(bins=bins)
data = [go.Scatter(x= germany.index,y=germany['Running Occurance'],name='Germany')]
#data2 = [go.Histogram(x= c,#germany['Running Occurance'],
#                      name='Germany')]
layout = go.Layout(title='scatter plot with pandas',
                   yaxis=dict(title='random distribution'), 
                   xaxis=dict(title='linspace'))
py.iplot(data, filename='pandas/basic-line-plot')#'basic-histogram'

In [82]:
grouped = rafo3r.groupby('Words')
#grouped.describe()
#display(rafo3r.head())

In [41]:
rafo3r['Occurance'].sort_values()[-10:] # top 10 values

43438     4219
26945     4219
182840    4219
202914    4219
587368    4219
546345    4219
244596    4219
26925     4219
59492     4219
341261    4219
Name: Occurance, dtype: int64

In [55]:
rafo3r['Position'] = rafo3r.index

occurances = rafo3r.groupby('Words').size()
mega_words = occurances.index[occurances >= 1000]
rafo3r.index = rafo3r['Words']
#sw = rafo3r.ix['Stop Word']
#sw.head()

In [83]:
rafo3r.head(1)

,Words,Stop Word,Occurance,Running Occurance,Position
Words,,,,,
william,william,False,12,1,0


In [90]:
def start_loc(x):
    return int(100 * min(x) / len(rafo3r))
def end_loc(x):
    return int(100 * max(x) / len(rafo3r))
rafo3r_pt = rafo3r[rafo3r['Stop Word'] == False].pivot_table(values='Position', 
            aggfunc=[len,start_loc,end_loc], index='Words')
rafo3r_pt.sort_values('len',ascending=False,inplace=True)
#this will be more useful once you add chapters
display(rafo3r_pt[:10])

,len,start_loc,end_loc
Words,,,
hitler,4219,0,99
german,2759,0,99
germany,1713,0,99
war,1591,0,99
army,1224,0,99
fuehrer,1076,0,99
british,1052,2,99
time,1034,0,99
berlin,1027,0,99


In [95]:
display (list(toc.index))

['B1',
 'Ch1',
 'Ch2',
 'Ch3',
 'Ch4',
 'B2',
 'Ch5',
 'Ch6',
 'Ch7',
 'Ch8',
 'B3',
 'Ch9',
 'Ch10',
 'Ch11',
 'Ch12',
 'Ch13',
 'Ch14',
 'Ch15',
 'Ch16',
 'Ch17',
 'B4',
 'Ch18',
 'Ch19',
 'Ch20',
 'Ch21',
 'Ch22',
 'Ch23',
 'Ch24',
 'Ch25',
 'Ch26',
 'B5',
 'Ch27',
 'Ch28',
 'Ch29',
 'B6',
 'Ch30',
 'Ch31',
 'Ch32',
 'Ch33']